# KENV

<a href=mailto:fuodorov1998@gmail.com>V. Fedorov</a>, <a href=mailto:nikdanila@bk.ru>D. Nikiforov</a>, <a href=http://www.inp.nsk.su/~petrenko/>A. Petrenko</a>, (Novosibirsk, 2019)

## Manual

In [1]:
import kenv as kv

Сan call for help:

In [2]:
#help(kv)

## Simulation

To obtain an envelope, you need to create a beam and an accelerator. Then load the simulation of this beam in this accelerator and apply tracking.

In [3]:
beam = kv.Beam(energy=2,
               current=2e3,
               radius=50e-3,
               x = 30e-3,
               y = 0,
               rp=55.0e-3,
               normalized_emittance=1000e-6)

Can print:

In [4]:
print(beam)

Beam parameters:
	Current	2000 A
	Energy	2.000 MeV
	Total momentum	2.458 MeV/c
	Rel. factor	4.914
	Radius x	50.0 mm
	Radius y	50.0 mm
	Radius x prime	55.0 mrad
	Radius y prime	55.0 mrad
	Horizontal centroid position	30.0 mm
	Vertical centroid position	0.0 mm
	Horizontal centroid angle	0.0 mrad
	Vertical centroid angle	0.0 mrad
	Larmor angle	0.0 rad
	Normalized emittance x	1000.0 mm*mrad
	Normalized emittance y	1000.0 mm*mrad



In [5]:
accelerator = kv.Accelerator(0.7, 15, 0.001)

In [6]:
Solenoids = [ 
    [ 0.95,       0.001,    'Bz.dat', 'Sol. 1' ],
    [ 2.1,        0.03,     'Bz.dat', 'Sol. 2' ],
    [ 2.9077,     0.001,    'Bz.dat', 'Sol. 3' ],
    [ 4.0024,     0.03,     'Bz.dat', 'Sol. 4' ],
    [ 5.642,      0.03,     'Bz.dat', 'Sol. 5' ],
    [ 6.760,      0.001,    'Bz.dat', 'Sol. 6' ],
]

In [7]:
Accels = [
    [ 7.456,       -0.9,     'Ez.dat',  'Cavity 3'],
    [ 8.838,       -0.9,     'Ez.dat',  'Cavity 4'],
    [ 10.220,      -0.9,     'Ez.dat',  'Cavity 5'],
    [ 11.602,      -0.9,     'Ez.dat',  'Cavity 6'],
    [ 12.984,      -0.9,     'Ez.dat',  'Cavity 7'],
    [ 14.366,      -0.9,     'Ez.dat',  'Cavity 8'],    
]

In [8]:
for   z0, B0, filename, name in Solenoids:
    accelerator.Bz_beamline[name] = kv.Element(z0, B0, filename, name, x=0.000, xp=0.003)
for   z0, E0, filename, name in Accels:
    accelerator.Ez_beamline[name] = kv.Element(z0, E0, filename, name)

In [9]:
accelerator.compile()

In [10]:
simulation = kv.Simulation(beam, accelerator)

In [11]:
simulation.track()

## Graphic with holoviews

### matplotlib

In [ ]:
import holoviews as hv
hv.extension('matplotlib')

%opts Layout [tight=True]
%output size=250 backend='matplotlib' fig='png'

%opts Area Curve [aspect=3 show_grid=True]
%opts Area  (alpha=0.25)
%opts Curve (alpha=0.5)
%opts Area.Beam [aspect=3 show_grid=True] (color='red' alpha=0.3)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dim_z  = hv.Dimension('z',  unit='m', range=(accelerator.start, accelerator.stop))
dim_z_zoom = hv.Dimension('z_zoom',  unit='m', range=(7, 8.0))
dim_Bz = hv.Dimension('Bz', unit='T', label='Bz')
dim_Ez = hv.Dimension('Ez', unit='MV/m', label='Ez')
dim_dEzdz = hv.Dimension('dEzdz', unit='$\mathrm{MV/m^2}$', label='$dE_z/dz$', range=(-5,+5))
dim_dEzdz_zoom = hv.Dimension('dEzdz_zoom', unit='$\mathrm{MV/m^2}$', label='$dE_z/dz$_zoom', range=(-8,+8))
dim_dBzdz = hv.Dimension('dBzdz', unit='T/m', label='$dB_z/dz$', range=(-0.22,+0.22))

dim_x = hv.Dimension('x', label="Coordinate", unit='mm', range=(-40,40))
dim_y = hv.Dimension('y', label="Coordinate", unit='mm', range=(-40,40))
dim_r = dim_x
dim_phase = hv.Dimension('phase', label="Larmor phase $\Psi$", unit='rad')

In [ ]:
z_Ez = hv.Area((accelerator.parameter,accelerator.Ez(accelerator.parameter)), kdims=dim_z, vdims=dim_Ez)
z_dEzdz = hv.Area((accelerator.parameter,-accelerator.dEzdz(accelerator.parameter)), kdims=dim_z, vdims=dim_dEzdz)
z_dEzdz_zoom = hv.Area((accelerator.parameter,-accelerator.dEzdz(accelerator.parameter)), kdims=dim_z_zoom, vdims=dim_dEzdz_zoom)
z_Bz = hv.Area((accelerator.parameter,accelerator.Bz(accelerator.parameter)), kdims=dim_z, vdims=dim_Bz)
z_dBzdz = hv.Area((accelerator.parameter,-accelerator.dBzdz(accelerator.parameter)), kdims=dim_z, vdims=dim_dBzdz)

z_r = hv.Area(((accelerator.parameter,simulation.envelope_x(accelerator.parameter)*1e3)), kdims=[dim_z], vdims=[dim_r], group='Beam')
z_x_centroid = hv.Curve(((accelerator.parameter,simulation.centroid_x(accelerator.parameter)*1e3)), kdims=[dim_z], vdims=[dim_x], group='Beam', label='x')
z_y_centroid = hv.Curve(((accelerator.parameter,simulation.centroid_y(accelerator.parameter)*1e3)), kdims=[dim_z], vdims=[dim_y], group='Beam', label='y')

z_larmor_phase = hv.Curve(((accelerator.parameter,simulation.larmor_angle(accelerator.parameter))), kdims=[dim_z], vdims=[dim_phase])


In [ ]:
(z_x_centroid*z_y_centroid+z_larmor_phase).cols(1)

In [ ]:
(z_Bz + z_Ez + z_dEzdz + z_dEzdz_zoom).cols(1)

## Astra vs KENV

In [ ]:
import pandas as pd

In [ ]:
%opts Curve.Astra (linestyle='--' color='black')
track = "input/0B_track_centroid/track_centroid.track.001"# Astra output file

cols = ['id1', 'id2', 'z', 'x', 'y', 'id4', 'id5', 'id6', 'id7']
df = pd.read_csv(track, header=None, delim_whitespace=True, names=cols, dtype='float32')

larmor = "input/0B_track_centroid/larmor_angle.dat"
cols_larmor = ['z', 'phase','id']
df_larmor = pd.read_csv(larmor, header=None, delim_whitespace=True, names=cols_larmor, dtype='float32')

field_dEz = "input/0B_track_centroid/dEz.dat"
cols_dEz = ['z', 'dEzdz', 'id']
df_dEz = pd.read_csv(field_dEz, header=None, delim_whitespace=True, names=cols_dEz, dtype='float32')

field_dBz = "input/0B_track_centroid/dBz.dat"
cols_dBz = ['z', 'dBzdz', 'id']
df_dBz = pd.read_csv(field_dBz, header=None, delim_whitespace=True, names=cols_dBz, dtype='float32')

z_x_astra = hv.Curve(df, kdims=[dim_z,dim_x], label='Astra', group = 'Astra')
z_y_astra = hv.Curve(df, kdims=[dim_z,dim_y], label='Astra', group = 'Astra')
z_larmor_astra = hv.Curve(df_larmor, kdims=[dim_z,dim_phase], label='Astra', group = 'Astra')
z_dEzdz_astra = hv.Curve(df_dEz, kdims=[dim_z,dim_dEzdz], label='Astra', group = 'Astra')
z_dBzdz_astra = hv.Curve(df_dBz, kdims=[dim_z,dim_dBzdz], label='Astra', group = 'Astra')


(z_x_astra*z_x_centroid*z_y_astra*z_y_centroid+z_larmor_phase*z_larmor_astra+z_dBzdz*z_dBzdz_astra+z_dEzdz*z_dEzdz_astra).cols(1)

## Difference 

In [ ]:

cols = ['id1', 'id2', 'z', 'x', 'y', 'id4', 'id5', 'id6', 'id7']
df = pd.read_csv(track, header=None, delim_whitespace=True, names=cols, dtype='float32')

import numpy as np
x_astra_array = df['x']
y_astra_array = df['y']
z_array = df['z']

In [ ]:
x_kenv_array = np.interp(z_array, accelerator.parameter, simulation.centroid_x(accelerator.parameter)*1e3)
y_kenv_array = np.interp(z_array, accelerator.parameter, simulation.centroid_y(accelerator.parameter)*1e3)

In [ ]:
delta_x = -x_kenv_array+x_astra_array
delta_y = -y_kenv_array+y_astra_array

In [ ]:
dim_x = hv.Dimension('x', label="Delta", unit='mm', range=(-4,4))
dim_y = hv.Dimension('y', label="Delta", unit='mm', range=(-4,4))


(hv.Curve(((z_array, delta_x)), kdims=[dim_z], vdims=[dim_x], label='$\delta x$')\
*hv.Curve(((z_array, delta_y)), kdims=[dim_z], vdims=[dim_y], label='$\delta y$')+z_dBzdz+z_dEzdz).cols(1)